In [2]:
# Dense.py 파일의 모듈을 불러옴
from Dense import DenseRetrievalModel, DenseRetrievalArguments, create_dataloader, train_dense, ContrastiveLoss, load_datasets
from transformers import AdamW, get_linear_schedule_with_warmup
import torch

# 하이퍼파라미터 설정
dense_args = DenseRetrievalArguments(
    data_route='/data/ephemeral/template_2/data/train_dataset',
    test_data_route='/data/ephemeral/data/test_dataset',
    model_name="klue/bert-base",
    num_negatives=1,
    margin=0.5
)

# 데이터셋 로드
train_dataset, validation_dataset, test_dataset = load_datasets(
    sample_size=None,  # 원본 크기의 데이터셋 로드
    data_path=dense_args.data_route
)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


중복 제거 후 학습 데이터셋 크기: 3340
중복 제거 후 검증 데이터셋 크기: 235
중복 제거 후 테스트 데이터셋 크기: 235


In [3]:
# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DenseRetrievalModel(model_name=dense_args.model_name).to(device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (init

In [4]:
# 데이터셋 형태 확인
print(train_dataset[0])

{'title': '미국 상원', 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05', 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?', 'id': 'mrc-1-000067', 'answers': {'answer_start': [2

In [5]:
# 전처리 단계
all_contexts = [doc['context'] for doc in train_dataset]
train_loader = create_dataloader(
    train_dataset=train_dataset,
    tokenizer=model.tokenizer,
    all_contexts=all_contexts,
    batch_size=16,
    shuffle=True,
    num_negatives=dense_args.num_negatives
)


In [6]:
optimizer = AdamW(model.parameters(), lr=3e-5)
total_steps = len(train_loader) * 3  # 3 epochs 기준
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
criterion = ContrastiveLoss(margin=dense_args.margin)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
#학습
epochs = 3  # 학습 에포크 수
train_dense(model, train_loader, optimizer, scheduler, criterion, device, epochs=epochs)


Epoch 1/3: 100%|██████████| 209/209 [03:54<00:00,  1.12s/it]


Epoch 1 - Loss: 0.0172


Epoch 2/3: 100%|██████████| 209/209 [03:54<00:00,  1.12s/it]


Epoch 2 - Loss: 0.0083


Epoch 3/3: 100%|██████████| 209/209 [03:54<00:00,  1.12s/it]

Epoch 3 - Loss: 0.0058


In [9]:
#모델 저장
model_save_path = "./retrieval_result/dense_retrieval_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"모델이 저장되었습니다: {model_save_path}")


모델이 저장되었습니다: ./retrieval_result/dense_retrieval_model.pth


In [12]:
from Dense import DenseRetrievalModel, encode_documents
import torch
import json
import numpy as np
from torch.utils.data import Dataset

# 모델 경로 설정 및 로드
model_path = "./retrieval_result/dense_retrieval_model.pth"  # 저장된 모델 경로

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "klue/bert-base"  # 학습에 사용한 모델 이름
model = DenseRetrievalModel(model_name=model_name).to(device)

# 저장된 모델 가중치 로드
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()  # 모델을 평가 모드로 전환 (임베딩 생성 시 필요)

print(f"모델 가중치가 {model_path}에서 로드되었습니다.")


# 임베딩 생성 및 저장)
# 문서 데이터 로드 (위키 문서 등)
# 커스텀 위키 데이터셋 
class WikiDataset(Dataset):
    def __init__(self, docs):
        self.docs = docs
        self.doc_ids = list(docs.keys())
        self.contexts = [docs[doc_id]['text'] for doc_id in self.doc_ids]

    def __len__(self):
        return len(self.doc_ids)

    def __getitem__(self, idx):
        return {
            'id': self.doc_ids[idx],
            'text': self.contexts[idx]
        }

# JSON 파일 로드
with open("/data/ephemeral/data/wikipedia_documents.json", 'r', encoding='utf-8') as f:
    wiki_docs = json.load(f)

# WikiDataset으로 변환
wiki_dataset = WikiDataset(wiki_docs)

# 문서 임베딩 생성
doc_embeddings, doc_ids, contexts = encode_documents(
    model=model,
    wiki_docs=wiki_dataset,  
    tokenizer=model.tokenizer,
    device=device,
    batch_size=32  # 필요에 따라 배치 크기를 조정
)

# 임베딩을 저장할 경로 설정
embedding_save_path = "./retrieval_result/Dense_embedding.npy"
np.save(embedding_save_path, doc_embeddings)

# 문서 ID와 내용을 저장
doc_ids_path = "./retrieval_result/doc_ids.json"
contexts_path = "./retrieval_result/contexts.json"
with open(doc_ids_path, 'w', encoding='utf-8') as f:
    json.dump(doc_ids, f)
with open(contexts_path, 'w', encoding='utf-8') as f:
    json.dump(contexts, f)

print(f"문서 임베딩과 문서 ID, 내용을 저장했습니다.")


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions

모델 가중치가 ./retrieval_result/dense_retrieval_model.pth에서 로드되었습니다.


Encoding Documents: 100%|██████████| 1895/1895 [09:24<00:00,  3.36it/s]


문서 임베딩과 문서 ID, 내용을 저장했습니다.
